In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import pandas as pd
import torch.optim as opt
import sys
import utils
import hyp
from utils import PrioritizedReplayBuffer 
from rdkit import Chem
from rdkit.Chem import QED, AllChem, rdFingerprintGenerator, Crippen, Descriptors, MACCSkeys
from rdkit.Chem.Crippen import MolLogP
from rdkit.Contrib.SA_Score import sascorer
from environment import Molecule
from torch.utils.tensorboard import SummaryWriter
import os
import subprocess
from tqdm import tqdm
import pickle
import catboost

In [9]:
def smiles_to_pdbqt(smiles: str, output_file: str = "./docking/ligant.pdbqt"):
    """Конвертирует SMILES в PDBQT через Open Babel."""
    # Создание молекулы из SMILES и добавление водородов
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    status = AllChem.EmbedMolecule(mol)
    if status == -1:
        raise RuntimeError("Не удалось сгенерировать 3D-структуру")
    
    # Сохранение во временный файл .mol
    temp_mol = "./docking/temp.mol"
    Chem.MolToMolFile(mol, temp_mol)
    if not os.path.exists(temp_mol):
        raise FileNotFoundError("Временный файл не создан")
    
    # Конвертация в PDBQT через Open Babel
    cmd = f"obabel {temp_mol} -O {output_file}"
    result = subprocess.run(
        cmd, 
        shell=True, 
        capture_output=True, 
        text=True
    )

    if result.returncode != 0:
        error_msg = f"Ошибка Open Babel:\n{result.stderr}"
        if "Invalid output format" in result.stderr:
            error_msg += "\nУбедитесь, что Open Babel установлен и добавлен в PATH"
        raise RuntimeError(error_msg)


    
    if not os.path.exists(output_file):
            raise FileNotFoundError(f"Файл {output_file} не создан")
    
    #os.remove(temp_mol)

def run_vina_docking(protein_pdbqt: str, ligand_pdbqt: str, center: tuple = (5 , 15, 50), size: tuple = ( 20, 20, 20)) -> float:
    """Запускает докинг и возвращает энергию связывания."""
    # Создание конфигурационного файла для Vina
    config = f"""
    receptor = {protein_pdbqt}
    ligand = {ligand_pdbqt}
    out = result.pdbqt
    center_x = {center[0]}
    center_y = {center[1]}
    center_z = {center[2]}
    size_x = {size[0]}
    size_y = {size[1]}
    size_z = {size[2]}
    exhaustiveness = 16
    cpu = 12
    """
    with open("./docking/config.txt", "w") as conf_file:
        conf_file.write(config)
    # Запуск AutoDock Vina
    with open("./docking/log.txt", "w") as log_file:
        result = subprocess.run(
            f"vina --config ./docking/config.txt",
            stdout=log_file,
            text=True,
            check=True,
            shell=True
        )
        if result.returncode != 0:
            print("Ошибка Vina:", result.stderr)
            return None
    os.remove("result.pdbqt")
    # Извлечение энергии связывания из лога
    with open("./docking/log.txt", "r") as f:
        log = f.read()
        affinity_values = []
        for line in log.split("\n"):
            if line.strip().startswith("1"):  # Первая строка с результатами
                parts = line.split()
                if len(parts) >= 2:
                    try:
                        affinity = float(parts[1])
                        affinity_values.append(affinity)
                    except ValueError:
                        continue
    
        # Возвращаем лучшую энергию
        if affinity_values:
            return min(affinity_values)
        else:
            print("Энергии связывания не найдены")
            return None

In [10]:
smiles_to_pdbqt("[H]C(N)c1c(N(C)C#N)nc2[nH]cc(N=O)c2c1-c1cc2c3c(n1)C(CN)(CN)C2(N=N)NC3=O")


In [11]:
run_vina_docking("./docking/GSK3-b.pdb", "./docking/ligant.pdbqt")

0.0

In [3]:
class NoisyLinear(nn.Module):
    """Noisy Linear Layer for exploration"""
    def __init__(self, in_features, out_features, std_init=0.4):
        super(NoisyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init
        
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weight_sigma = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias_mu = nn.Parameter(torch.Tensor(out_features))
        self.bias_sigma = nn.Parameter(torch.Tensor(out_features))
        
        self.register_buffer('weight_epsilon', torch.Tensor(out_features, in_features))
        self.register_buffer('bias_epsilon', torch.Tensor(out_features))
        
        self.reset_parameters()
        self.reset_noise()
    
    def reset_parameters(self):
        mu_range = 1 / np.sqrt(self.in_features)
        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(self.std_init / np.sqrt(self.in_features))
        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(self.std_init / np.sqrt(self.out_features))
    
    def reset_noise(self):
        epsilon_in = self.scale_noise(self.in_features)
        epsilon_out = self.scale_noise(self.out_features)
        
        self.weight_epsilon.copy_(epsilon_out.outer(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)
    
    def forward(self, x):
        if self.training:
            weight = self.weight_mu + self.weight_sigma * self.weight_epsilon
            bias = self.bias_mu + self.bias_sigma * self.bias_epsilon
            return F.linear(x, weight, bias)
        else:
            return F.linear(x, self.weight_mu, self.bias_mu)
    
    @staticmethod
    def scale_noise(size):
        x = torch.randn(size)
        return x.sign().mul_(x.abs().sqrt_())

class RainbowDQN(nn.Module):
    """Rainbow DQN Network with Dueling Architecture and Distributional RL"""
    def __init__(self, input_length, output_length, atoms=51, v_min=-10, v_max=10):
        super(RainbowDQN, self).__init__()
        self.atoms = atoms
        self.v_min = v_min
        self.v_max = v_max
        self.output_length = output_length
        
        # Feature extraction
        self.linear_1 = NoisyLinear(input_length, 1024)
        self.linear_2 = NoisyLinear(1024, 512)
        
        # Dueling streams
        self.value_stream = nn.Sequential(
            NoisyLinear(512, 128),
            nn.ReLU(),
            NoisyLinear(128, atoms)
        )
        
        self.advantage_stream = nn.Sequential(
            NoisyLinear(512, 128),
            nn.ReLU(),
            NoisyLinear(128, output_length * atoms)
        )
            
    def forward(self, x):
       
        x = F.relu(self.linear_1(x))
        x = F.relu(self.linear_2(x))
        
        value = self.value_stream(x).view(-1, 1, self.atoms)
        advantage = self.advantage_stream(x).view(-1, self.output_length, self.atoms)
        
        q_dist = value + advantage - advantage.mean(dim=1, keepdim=True)
        return F.softmax(q_dist, dim=2)
    
    def reset_noise(self):
        for module in self.modules():
            if isinstance(module, NoisyLinear):
                module.reset_noise()
    
    def get_q_values(self, x):
        with torch.no_grad():
            dist = self.forward(x)
            support = torch.linspace(self.v_min, self.v_max, self.atoms).to(x.device)
            q_values = (dist * support).sum(dim=2)
            return q_values


In [5]:

REPLAY_BUFFER_CAPACITY = hyp.replay_buffer_size

def predict_activity(smiles: str) -> dict:
    """
    Предсказывает pIC50 и IC50 для молекулы по SMILES-строке.
    
    Аргументы:
        smiles (str): SMILES-представление молекулы
        
    Возвращает:
        dict: Словарь с предсказаниями pIC50 и IC50
        или сообщение об ошибке
    """
    MODEL = None
    COLUMNS = None
    
    # Загрузка модели и списка колонок при первом вызове
    if MODEL is None:
        try:
            with open('./submodel/final_catboost_model.pkl', 'rb') as f:
                MODEL = pickle.load(f)
            with open('./submodel/descriptor_columns.pkl', 'rb') as f:
                COLUMNS = pickle.load(f)
        except Exception as e:
            return {"error": f"Ошибка загрузки модели: {str(e)}"}
    
    # Преобразование SMILES в молекулярный объект
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {"error": "Невалидный SMILES"}
    
    try:
        # Вычисление обычных дескрипторов
        desc_calc = {name: func for name, func in Descriptors.descList}
        row = {}
        
        # Разделение колонок на обычные и MACCS
        regular_cols = [col for col in COLUMNS if not col.startswith('maccs_')]
        maccs_cols = [col for col in COLUMNS if col.startswith('maccs_')]
        
        # Вычисление химических дескрипторов
        for col in regular_cols:
            if col in desc_calc:
                row[col] = desc_calc[col](mol)
            else:
                return {"error": f"Неизвестный дескриптор: {col}"}
        
        # Генерация MACCS-фингерпринтов
        fp = MACCSkeys.GenMACCSKeys(mol)
        for col in maccs_cols:
            bit_idx = int(col.split('_')[1])
            row[col] = 1 if fp.GetBit(bit_idx) else 0
        

        # Создание DataFrame с сохранением порядка колонок
        input_data = pd.DataFrame([row], columns=COLUMNS)
        
        # Предсказание pIC50
        pIC50 = MODEL.predict(input_data)[0]# Здесь модель обучалась на pIC50
        
        # Конвертация в IC50 (в наномолях)
        IC50_M = 10 ** (-pIC50)  # в молях
        IC50_nM = IC50_M * 1e9   # в наномолях

        # Расчет AlogP
        alogp = MolLogP(mol)
        
        return {
            "pIC50": round(pIC50, 6),
            "IC50": IC50_nM,
            "AlogP": round(alogp, 6)
        }
        
    except Exception as e:
        return {"error": f"Ошибка предсказания: {str(e)}"}

class QEDRewardMolecule(Molecule):
    
    def __init__(self, discount_factor, **kwargs):
        
        super(QEDRewardMolecule, self).__init__(**kwargs)
        self.discount_factor = discount_factor

    def _reward(self):
        
        molecule = Chem.MolFromSmiles(self._state)
        if molecule is None:
            return 0.0
            
        activity = predict_activity(self._state)
        IC50 = float(activity['IC50'])
        AlogP = float(activity['AlogP'])
        qed = QED.qed(molecule)

        if IC50 > 200:
            reward = -IC50
        else:
            reward = (100/IC50 + AlogP/6 + qed) * self.discount_factor 
        
        return reward
        
    def _goal_reached(self):
        molecule = Chem.MolFromSmiles(self._state)
        if molecule is None:
            return 0.0

        activity = predict_activity(self._state)
        IC50 = float(activity['IC50'])
        AlogP = float(activity['AlogP'])
        qed = QED.qed(molecule)

        return AlogP > 3 and IC50 < 30 and qed > 0.5 and self._counter >= 4

In [6]:
 class RainbowAgent:
    def __init__(self, input_length, output_length, device, atoms=51, v_min=-10, v_max=10):
        self.device = device
        self.atoms = atoms
        self.v_min = v_min
        self.v_max = v_max
        self.delta_z = (v_max - v_min) / (atoms - 1)
        self.support = torch.linspace(v_min, v_max, atoms).to(device)
        
        # Main and target networks
        self.dqn = RainbowDQN(input_length, output_length, atoms, v_min, v_max).to(device)
        self.target_dqn = RainbowDQN(input_length, output_length, atoms, v_min, v_max).to(device)
        self.target_dqn.load_state_dict(self.dqn.state_dict())
        
        self.replay_buffer = PrioritizedReplayBuffer(hyp.replay_buffer_size)
        self.optimizer = getattr(opt, hyp.optimizer)(self.dqn.parameters(), lr=hyp.learning_rate)
        self.times_of_update = 0
    
    def get_action(self, observations, epsilon_threshold):
        if np.random.uniform() < epsilon_threshold:
            return np.random.randint(0, observations.shape[0])
        
        observations = observations.to(self.device)
        with torch.no_grad():
            q_values = self.dqn.get_q_values(observations).cpu()
        return torch.argmax(q_values).item()
    
    def update_params(self, batch_size, gamma, polyak):
        if len(self.replay_buffer) < batch_size:
            return None
        
        # Sample from prioritized replay buffer
        samples, indices, weights = self.replay_buffer.sample(batch_size)
        weights = weights.to(self.device)
        
        # Unpack batch
        states, _, rewards, next_states, dones = zip(*samples)
        states = torch.stack([torch.FloatTensor(s) for s in states]).to(self.device)
        next_states = torch.stack([torch.FloatTensor(ns) for ns in next_states]).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        # Distributional DQN update
        with torch.no_grad():
            # Next state distribution
            next_dist = self.target_dqn(next_states)
            next_q = (next_dist * self.support).sum(2)
            next_actions = next_q.argmax(1)
            
            # Project next distribution
            next_dist = next_dist[range(batch_size), next_actions]
            rewards = rewards.unsqueeze(1).expand_as(next_dist)
            dones = dones.unsqueeze(1).expand_as(next_dist)
            support = self.support.unsqueeze(0).expand_as(next_dist)
            
            Tz = rewards + gamma * support * (1 - dones)
            Tz = Tz.clamp(self.v_min, self.v_max)
            b = (Tz - self.v_min) / self.delta_z
            l = b.floor().long()
            u = b.ceil().long()
            
            offset = torch.linspace(0, (batch_size - 1) * self.atoms, batch_size).long()\
                .unsqueeze(1).expand(batch_size, self.atoms).to(self.device)
            
            proj_dist = torch.zeros(next_dist.size()).to(self.device)
            proj_dist.view(-1).index_add_(0, (l + offset).view(-1), 
                                          (next_dist * (u.float() - b)).view(-1))
            proj_dist.view(-1).index_add_(0, (u + offset).view(-1), 
                                          (next_dist * (b - l.float())).view(-1))

        
        # Current state distribution
        dist = self.dqn(states)
        actions = torch.argmax(self.dqn.get_q_values(states), dim=1)
        dist = dist[range(batch_size), actions]
        
        # Calculate loss
        log_dist = torch.log(dist.clamp(min=1e-5))
        loss = - (proj_dist * log_dist).sum(1)
        weighted_loss = (weights * loss).mean()
        
        # Backpropagation
        self.optimizer.zero_grad()
        weighted_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.dqn.parameters(), 10)
        self.optimizer.step()
        
        # Update priorities
        priorities = loss.detach().cpu().numpy() + 1e-5
        self.replay_buffer.update_priorities(indices, priorities)
        
        # Update target network
        if self.times_of_update % hyp.update_interval == 0:
            with torch.no_grad():
                for param, target_param in zip(self.dqn.parameters(), self.target_dqn.parameters()):
                    target_param.data.copy_(polyak * target_param.data + (1 - polyak) * param.data)
        
        self.times_of_update += 1
        self.dqn.reset_noise()
        self.target_dqn.reset_noise()
        
        return weighted_loss.item()

In [7]:
# Инициализация
TENSORBOARD_LOG = True
TB_LOG_PATH = "./runs/dqn/run2"
episodes = 0
iterations = 10000
num_updates_per_it = 1

initmols = pd.read_csv("./InitMols.csv", sep=';')["Smiles"].to_numpy()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
environment = QEDRewardMolecule(
    discount_factor=hyp.discount_factor,
    atom_types=set(hyp.atom_types),
    init_mols = initmols,
    allow_removal=hyp.allow_removal,
    allow_no_modification=hyp.allow_no_modification,
    allow_bonds_between_rings=hyp.allow_bonds_between_rings,
    allowed_ring_sizes=set(hyp.allowed_ring_sizes),
    max_steps=hyp.max_steps_per_episode,
)

# Rainbow Agent вместо DQN
agent = RainbowAgent(
    input_length=hyp.fingerprint_length + 1,
    output_length=1,
    device=device,
    atoms=51,
    v_min=-10,
    v_max=10
)

# Загрузка весов
# agent.dqn.load_state_dict(torch.load("best_weights.pt"))

if TENSORBOARD_LOG:
    writer = SummaryWriter(TB_LOG_PATH)

environment.initialize()
eps_threshold = 1
best_reward = -1000

In [ ]:
for it in tqdm(range(iterations)):
    steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
    valid_actions = list(environment.get_valid_actions())
    
    # Подготовка наблюдений
    observations = np.vstack([
        np.append(
            utils.get_fingerprint(act, hyp.fingerprint_length, hyp.fingerprint_radius),
            steps_left
        ) for act in valid_actions
    ])
    
    observations_tensor = torch.Tensor(observations)
    
    # Выбор действия
    action_idx = agent.get_action(observations_tensor, max(0.1, eps_threshold))
    action = valid_actions[action_idx]
    
    # Шаг среды
    result = environment.step(action)
    next_state, reward, done = result
    
    # Сохранение в буфер (с n-step)
    action_fingerprint = np.append(
        utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
        steps_left
    )

    steps_left_next = steps_left - 1 if not done else 0
    next_state_fp = np.append(
        utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
        steps_left_next
    )
    

    # Добавляем переход в буфер 
    agent.replay_buffer.add((
        action_fingerprint, 
        action_idx, 
        reward, 
        next_state_fp, 
        float(done)
    ), n_step=3, gamma=hyp.discount_factor)
    
    # Обновление модели
    if it % hyp.update_interval == 0 and len(agent.replay_buffer) > hyp.batch_size:
        loss = agent.update_params(hyp.batch_size, hyp.gamma, hyp.polyak)
        
        if TENSORBOARD_LOG and loss is not None:
            writer.add_scalar("training/loss", loss, it)
    
    # Обработка завершения эпизода
    if done:
        final_reward = reward
        
        if TENSORBOARD_LOG:
            writer.add_scalar("episode/reward", final_reward, episodes)
            writer.add_scalar("episode/epsilon", eps_threshold, episodes)
        
        # Логирование и сохранение лучшей модели
        if episodes % 10 == 0:
            print(f"Episode {episodes}, Reward: {final_reward:.2f}, Best Reward: {best_reward:.2f}, Eps: {eps_threshold:.3f}")
            mol = Chem.MolFromSmiles(environment._state)
            if mol:
                activity = predict_activity(environment._state)
                IC50 = float(activity['IC50'])
                AlogP = float(activity['AlogP'])
                qed = QED.qed(mol)

                sa = sascorer.calculateScore(mol)
                print(f"  QED: {qed:.3f}, IC50: {IC50:.3f},  AlogP: {AlogP:.3f}, Molecule: {environment._state}")
                
        if final_reward > best_reward:
            torch.save(agent.dqn.state_dict(), 'best_weights.pt')
            print(f"Saved best model with reward: {final_reward:.2f}")
            best_reward = final_reward
        
        # Сброс среды и уменьшение epsilon
        environment.initialize()
        eps_threshold = max(hyp.epsilon_end, eps_threshold * hyp.gamma)
        episodes += 1

# Закрытие логгера
if TENSORBOARD_LOG:
    writer.close()

  0%|▏                                                                              | 29/10000 [00:02<19:39,  8.45it/s]

Episode 0, Reward: 0.76, Eps: 0.800


  0%|▏                                                                              | 30/10000 [00:03<37:22,  4.45it/s]

  QED: 0.124, IC50: 177.815,  AlogP: 0.495, Molecule: [H]C(N)c1c(N(C)C#N)nc2[nH]cc(N=O)c2c1-c1cc2c3c(n1)C(CN)(CN)C2(N=N)NC3=O
Saved best model with reward: 0.76


  3%|██▌                                                                           | 330/10000 [00:52<17:43,  9.09it/s]

...

 15%|███████████▊                                                                 | 1530/10000 [04:43<30:38,  4.61it/s]

Episode 50, Reward: -366.87, Eps: 0.655
  QED: 0.115, IC50: 366.873,  AlogP: 0.984, Molecule: [CH]C(=O)C1N=C(OC#CO)CC12C(=C)N2c1nc2c3c4nn2c(N=Cc2c5c6c([n+]([O-])c2-5)C(C)=C6)c1N4CC3(C#N)CN


 18%|██████████████                                                               | 1830/10000 [05:46<30:47,  4.42it/s]

Episode 60, Reward: 1.49, Eps: 0.629
  QED: 0.099, IC50: 86.316,  AlogP: 1.475, Molecule: N#Cc1nn2nc3c4c5c6c(F)c7ooc8nc(N9C%10=C(C=NN9CN)C9=CC%11=C%10C9%11OO)nc(c8c74)c3c2c5c1C6(N)N
Saved best model with reward: 1.49


 20%|███████████████▎                                                             | 1994/10000 [06:21<27:45,  4.81it/s]

In [ ]:
generated_molecules = []
num_molecules_to_generate = 100
agent.dqn.eval()
eps_threshold = 0.03

for it in range(num_molecules_to_generate):
    done = False
    environment.initialize()
    while not done:
        steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
        valid_actions = list(environment.get_valid_actions())
    
        observations = np.vstack(
            [
                np.append(
                    utils.get_fingerprint(
                        act, hyp.fingerprint_length, hyp.fingerprint_radius
                    ),
                    steps_left,
                )
                for act in valid_actions
            ]
        ) 
    
        observations_tensor = torch.Tensor(observations)
        a = agent.get_action(observations_tensor, eps_threshold)
        action = valid_actions[a]
        result = environment.step(action)
    
        action_fingerprint = np.append(
            utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
            steps_left,
        )
    
        next_state, reward, done = result
        steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
    
        next_state = utils.get_fingerprint(
            next_state, hyp.fingerprint_length, hyp.fingerprint_radius
        )  
    
        action_fingerprints = np.vstack(
            [
                np.append(
                    utils.get_fingerprint(
                        act, hyp.fingerprint_length, hyp.fingerprint_radius
                    ),
                    steps_left,
                )
                for act in environment.get_valid_actions()
            ]
        )
        #print(environment._state)
        print(reward)
        
    generated_molecules.append(environment._state)
    print(generated_molecules[-1])

In [ ]:
generated_molecules

In [ ]:
from rdkit import Chem

valid_smiles = []
for smi in generated_molecules:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        valid_smiles.append(smi)

print(f"Сгенерировано валидных молекул: {len(valid_smiles)}/{len(generated_molecules)}")